# Eat Safe, Love

## Notebook Set Up

In [1]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
db = mongo['uk_foods']
collection_names = db.list_collection_names()
print(collection_names)


['establishments']


In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [6]:
import pandas as pd
# Find the establishments with a hygiene score of 20
query = {"RatingValue": 20}

# Use count_documents to display the number of documents in the result
count_20_hygiene = establishments.count_documents(query)
print("Number of establishments with a hygiene score of 20:", count_20_hygiene)

# Display the first document in the results using pprint
first_document_20_hygiene = establishments.find_one(query)
print("First document with a hygiene score of 20:")
pprint(first_document_20_hygiene)

Number of establishments with a hygiene score of 20: 0
First document with a hygiene score of 20:
None


In [7]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df_20_hygiene = pd.DataFrame(list(cursor))
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df_20_hygiene))
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_20_hygiene.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [8]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = query = {"LocalAuthorityName": "London", "RatingValue": {"$gte": 4}}

# Use count_documents to display the number of documents in the result
count_establishments_london = establishments.count_documents(query)
print("Number of establishments in London with a RatingValue greater than or equal to 4:", count_establishments_london)
# Display the first document in the results using pprint
first_document_london = establishments.find_one(query)
print("First document in London with a RatingValue greater than or equal to 4:")
pprint(first_document_london)

Number of establishments in London with a RatingValue greater than or equal to 4: 0
First document in London with a RatingValue greater than or equal to 4:
None


In [9]:
# Convert the result to a Pandas DataFrame
cursor = establishments.find(query)
df_establishments_london = pd.DataFrame(list(cursor))
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df_establishments_london))
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df_establishments_london.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [12]:
import math 
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

degree_search = 0.01
latitude = 51.4769 
longitude = 0.0005

query = {
    "RatingValue": 5,
    "Latitude": {"$gte": latitude - degree_search, "$lte": latitude + degree_search},
    "Longitude": {"$gte": longitude - degree_search, "$lte": longitude + degree_search}
}
sort = [("HygieneScore", 1)]
# Execute the query to find the top 5 establishments
cursor = establishments.find(query).sort(sort).limit(5)
# Print the results
print("Top 5 establishments with a RatingValue rating of 5, sorted by lowest hygiene score and nearest to Penang Flavours:")
for index, document in enumerate(cursor, start=1):
    print(f"{index}. Name: {document['name']}, Hygiene Score: {document['HygieneScore']}, Distance: {calculate_distance(penang_flavours_latitude, penang_flavours_longitude, document['Latitude'], document['Longitude'])} km")

Top 5 establishments with a RatingValue rating of 5, sorted by lowest hygiene score and nearest to Penang Flavours:


In [13]:

result_list = list(cursor)

# Convert the list of dictionaries to a Pandas DataFrame
df = pd.DataFrame(result_list)

# Display the DataFrame
print(df)

Empty DataFrame
Columns: []
Index: []


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [14]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
    {"$match": {"HygieneScore": 0}},
    {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
    {"$sort": {"count": -1}}
]

# Execute the pipeline
result = list(establishments.aggregate(pipeline))

# Print the number of documents in the result
total_establishments_hygiene_0 = sum(item['count'] for item in result)
print("Total establishments with a hygiene score of 0:", total_establishments_hygiene_0)


Total establishments with a hygiene score of 0: 0


In [15]:
# Convert the result to a Pandas DataFrame
df = pd.DataFrame(result)
# Display the number of rows in the DataFrame
print("Number of rows in the DataFrame:", len(df))
# Display the first 10 rows of the DataFrame
print("First 10 rows of the DataFrame:")
print(df.head(10))

Number of rows in the DataFrame: 0
First 10 rows of the DataFrame:
Empty DataFrame
Columns: []
Index: []
